# Match Meise Collectors to Wikidata Items Using *Nearest Neighbour*

In this example we add `eventDate` of the source data, when the sample was collected, to have a time reference, when the collector was alive. 

Basically we attempt a match of `canonical_string` of WikiData to `canonical_string` of the collectors (in this case the names were parsed beforehand into single names using <https://libraries.io/rubygems/dwc_agent>)

TODO:

- review code: match when full collector name was given instead of an abbreviated collector name
- review code: evaluate if multiple names (WikiData or collector data) are found
- match also with time periode of work (WikiData) ⇌ created time of the herbarium sheet (if no other life time data are available)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
import pprint, time, os
wikidata = pd.read_csv("data/wikidata_persons_botanists_20230703_1352.csv", index_col=0, low_memory=False)

wikidata.head()

,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768.0,1826.0,NaN,NaN,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818.0,1904.0,NaN,NaN,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718.0,1790.0,NaN,NaN,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805.0,1860.0,NaN,NaN,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808.0,1880.0,NaN,NaN,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# Create data frame with unique canonical strings 
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()

wd_matchtest

# colls = list(wikidata.columns)

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


In [3]:
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

wd_matchtest_fullnames


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


### Load Collectors Data Set

Data sources:

- Jupyter Notebook for [create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`


In [4]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/meise_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("./data/Meise_doi-10.15468-dl.ax9zkh/occurrence_recordedBy_eventDate_occurrenceIDs_20230830_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

# Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
#  work around: use datetime or using pd.Periode(…)
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(lambda x: pd.Period(x, freq='ms'))

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

convert eventDate_mean to pd.Period(...) in collectors
convert eventDate_min to pd.Period(...) in collectors
convert eventDate_max to pd.Period(...) in collectors


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
13153,A,Buto,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000
14717,A,C.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000,2008-09-01 00:00:00.000
16195,A,Carl G.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000
41495,A,H.M.L.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
57340,A,L. Do,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23815,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,6,http://www.botanicalcollections.be/specimen/BR...,1959-01-01 04:00:00.000,1958-08-13 00:00:00.000,1960-06-23 00:00:00.000
56141,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000
112669,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000
87661,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000


#### Check Composition of Parsed Collector Data

In [5]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (17613 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
13153,A,Buto,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000
16195,A,Carl G.,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000
88404,A,Scha,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
91338,A,Sti,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1852-01-01 00:00:00.000,1852-01-01 00:00:00.000,1852-01-01 00:00:00.000
61273,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91039,Čermak,Stan,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1926-05-01 00:00:00.000,1926-05-01 00:00:00.000,1926-05-01 00:00:00.000
91035,Čermák,Stan,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1925-10-12 00:00:00.000,1925-10-12 00:00:00.000,1925-10-12 00:00:00.000
93146,Şes,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,1,http://www.botanicalcollections.be/specimen/BR...,1972-06-13 00:00:00.000,1972-06-13 00:00:00.000,1972-06-13 00:00:00.000
93147,Şesan-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,9,http://www.botanicalcollections.be/specimen/BR...,1972-04-06 08:00:00.000,1971-07-12 00:00:00.000,1973-06-12 00:00:00.000


In [6]:
# check the parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    print(test_collectors.head())


----------------------------------------
show names with **particle** found 7986 records:

              family      given suffix particle  dropping_particle  nick  \
21082              A        NaN    NaN       Da                NaN   NaN   
104479  A Longhridge         G.    NaN    asper                NaN   NaN   
12132       A Robyns         Br    NaN       in                NaN   NaN   
61273     A. Brabant  Lochenies    NaN       in                NaN   NaN   
32511   A. Chevalier     Fleury    NaN       in                NaN   NaN   

       appellation title  occurrenceID_count  \
21082          NaN   NaN                   1   
104479         NaN   NaN                   1   
12132          NaN   NaN                   1   
61273          NaN   NaN                   1   
32511          NaN   NaN                   2   

                                 occurrenceID_firstsample  \
21082   http://www.botanicalcollections.be/specimen/BR...   
104479  http://www.botanicalcollections.

Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [7]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)
collectors.tail()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
23815,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,"Žertová, A.",6,http://www.botanicalcollections.be/specimen/BR...,1959-01-01 04:00:00.000,1958-08-13 00:00:00.000,1960-06-23 00:00:00.000
56141,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000
112669,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000
87661,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000
112672,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,"Т. Королева. V. Petrovsky, В.Пертовский",1,http://www.botanicalcollections.be/specimen/BR...,1973-06-24 00:00:00.000,1973-06-24 00:00:00.000,1973-06-24 00:00:00.000


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

collectors_unique

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,56,http://www.botanicalcollections.be/specimen/BR...,1898-01-28 14:12:18.462,1840-01-01 00:00:00.000,1995-10-26 00:00:00.000
1,A Longhridge,G.,NaN,asper,NaN,NaN,NaN,NaN,"A Longhridge, G.",1,http://www.botanicalcollections.be/specimen/BR...,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000
2,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000
3,A',NaN,NaN,NaN,NaN,NaN,NaN,NaN,A',1,http://www.botanicalcollections.be/specimen/BR...,1896-08-06 00:00:00.000,1896-08-06 00:00:00.000,1896-08-06 00:00:00.000
4,A'hlbon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'hlbon,1,http://www.botanicalcollections.be/specimen/BR...,1864-05-01 00:00:00.000,1864-05-01 00:00:00.000,1864-05-01 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66935,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000,1958-05-20 00:00:00.000
66936,Žertová,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Žertová, A.",20,http://www.botanicalcollections.be/specimen/BR...,1959-04-19 20:00:00.000,1958-07-09 00:00:00.000,1963-06-27 00:00:00.000
66937,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000,1850-07-09 00:00:00.000
66938,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000,1975-08-29 00:00:00.000


In [9]:
# show collectors with highest occurrenceID_collectors_count
collectors_unique.sort_values(by=['occurrenceID_collectors_count', 'family'], ascending=[False, True]).head(10)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
62148,Vanderyst,H.,NaN,NaN,NaN,NaN,NaN,NaN,"Vanderyst, H.",41259,http://www.botanicalcollections.be/specimen/BR...,1914-09-09 04:44:14.709,1819-06-07 00:00:00.000,1990-04-01 00:00:00.000
2160,Aptroot,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aptroot, A.",32921,http://www.botanicalcollections.be/specimen/BR...,1989-07-24 07:01:45.468,1874-04-03 00:00:00.000,2022-08-03 00:00:00.000
37673,Louis,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Louis, J.",30727,http://www.botanicalcollections.be/specimen/BR...,1938-05-08 00:55:43.006,1875-08-01 00:00:00.000,1987-08-29 00:00:00.000
62058,Vanden Berghen,C.,NaN,NaN,NaN,NaN,NaN,NaN,"Vanden Berghen, C.",29241,http://www.botanicalcollections.be/specimen/BR...,1967-06-16 14:58:50.174,1856-04-03 00:00:00.000,2000-01-20 00:00:00.000
39124,Malaisse,F.,NaN,NaN,NaN,NaN,NaN,NaN,"Malaisse, F.",26240,http://www.botanicalcollections.be/specimen/BR...,1996-04-08 05:10:00.543,1802-01-01 00:00:00.000,2017-01-09 00:00:00.000
16290,Duvigneaud,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Duvigneaud, J.",23751,http://www.botanicalcollections.be/specimen/BR...,1975-08-27 06:43:43.520,1874-06-26 00:00:00.000,2021-09-15 00:00:00.000
12146,Coppejans,E.,NaN,NaN,NaN,NaN,NaN,NaN,"Coppejans, E.",23015,http://www.botanicalcollections.be/specimen/BR...,1995-07-08 23:20:34.632,1967-01-01 00:00:00.000,2014-03-13 00:00:00.000
14319,Delvosalle,L.,NaN,NaN,NaN,NaN,NaN,NaN,"Delvosalle, L.",22529,http://www.botanicalcollections.be/specimen/BR...,1963-10-15 04:43:01.221,1900-06-24 00:00:00.000,2013-06-01 00:00:00.000
59206,Symoens,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Symoens, J.",22073,http://www.botanicalcollections.be/specimen/BR...,1966-05-16 18:50:07.066,1848-06-01 00:00:00.000,2017-01-09 00:00:00.000
2485,Arts,T.,NaN,NaN,NaN,NaN,NaN,NaN,"Arts, T.",21420,http://www.botanicalcollections.be/specimen/BR...,1989-04-12 02:03:50.513,1888-03-15 00:00:00.000,2000-11-26 00:00:00.000


In [10]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [11]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))

(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(1835-1906), G.A.F.E. = (1835-1906), Gustav Adolf Ferdinand Eichler'
'(1873-1926), S.S. = (1873-1926), Søren Sørensen'
'(1888–1973), G.A. = (1888–1973), Georges André'
'(1904-1990), J.J. = (1904-1990), Johannes Johannessen'


In [12]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD', r'', string)
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [13]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))

Show ngram examples:
- simple name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- data from collectors: [' A ', 'A L', ' Lo', 'Lon', 'ong', 'ngh', 'ghr', 'hri', 'rid', 'idg', 'dge', 'ge ', 'e G', ' G ']
- data from match-test: [' Wa', 'Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' Oh', 'Oh ']
- data from match-test (full name): [' 18', '188', '888', '881', '819', '197', '973', '73 ', '3 G', ' Ge', 'Geo', 'eor', 'org', 'rge', 'ges', 'es ', 's A', ' An', 'And', 'ndr', 'dr ']


Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (Meise) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [14]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step


def getNearestNeighbour(query, this_vectorizer, this_nbrs_data):
    """Calculate the k-nearest distance for query data using package scikit-learn


    @param query: DataFrame the query data to vectorize and transform
    @param this_vectorizer: the vectorizer of TfidfVectorizer
    @param this_nbrs_data: the data of NearestNeighbors calculations
    @return: (distances, indices) distances and indices
    @rtype (int, int)
    """
    queryTFIDF_ = this_vectorizer.transform(query)
    distances, indices = this_nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: Number of neighbors required for each sample by default for :meth:`kneighbors` queries (originally 5).

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """

    import time
    start = time.time()
    query_data = set(query_data)
    # convert list to set for better performance

    print('Vectorizing data. This may take a while...')
    # vectorize wikidata names
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_vector_data = vectorizer.fit_transform(match_data
        # wd_matchtest['canonical_string']
    )
    nbrs_data = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1).fit(tfidf_vector_data)
    duration = time.time() - start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    print('Getting nearest neighbours of %s data with %s neighbor sample(s)...' % (len(query_data), n_neighbors))
    distances, indices = getNearestNeighbour(query_data, vectorizer, nbrs_data)
    duration = time.time() - start
    print('Completed after %s s' % duration)

    query_data = list(query_data)  # convert back to list

    print('Finding matches build new data frame ...')
    matches = []
    for i, j in enumerate(indices):
        temp = [query_data[i], match_data.values[j][0], round(distances[i][0], 2)]
        matches.append(temp)

    duration = time.time() - start
    print('Building matches done after %s s' % duration)
    matches = pd.DataFrame(
        matches,
        columns=['namematch_source_data', 'namematch_resource_data', 'namematch_distance']
    )

    print('Done')
    return matches


In [15]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
print("Calculate matching for **abbrevated** names separately …")
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

matches

Calculate matching for **abbrevated** names separately …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.1185147762298584 s
Getting nearest neighbours of 54016 data with 5 neighbor sample(s)...
Completed after 336.88155031204224 s
Finding matches build new data frame ...
Building matches done after 337.43837785720825 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,35173,"Steinberg, C.","Steinberg, C.",0.00
1,34493,"Weiss, E.","Weiss, E.",0.00
2,34495,"Cicioni, G.","Cicioni, G.",0.00
3,34508,"Ryan, E.","Ryan, E.",0.00
4,34515,"Pawłowska, S.","Pawłowska, S.",0.00
...,...,...,...,...
54011,28205,Qcoenafg,"Proença, S.L.",1.24
54012,45300,"Cercle Scientifique, H.Lücher","Manrique, H.F.",1.25
54013,28734,Onzichtbaar,"Saar, M.",1.25
54014,10490,"Oluadnunle, A.","Plée, A.",1.25


In [16]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
print("Calculate matching for **full** names separately …")
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

matches_fullnames


Calculate matching for **full** names separately …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.9597718715667725 s
Getting nearest neighbours of 12924 data with 5 neighbor sample(s)...
Completed after 188.48776197433472 s
Finding matches build new data frame ...
Building matches done after 188.59539675712585 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,6461,"Bowie, James","Bowie, James",0.00
1,5357,"Razanatsoa, Jacqueline","Razanatsoa, Jacqueline",0.00
2,5324,"Bonar, Lee","Bonar, Lee",0.00
3,5320,"Delessert, Adolphe","Delessert, Adolphe",0.00
4,5315,"Petenatti, Elisa M.","Petenatti, Elisa M",0.00
...,...,...,...,...
12919,9722,"Pavia, Lab Crypt","Crystal, Fergus",1.26
12920,11732,"Onleesbaar, Monvoisins","Wiesbaur, Johann",1.26
12921,12923,"Liunaei, Die natiV.","Rajaei, P",1.26
12922,1330,"ClJunojum, Neopresbyter","Catesby, Mark",1.27


### Create Output Results

Combine the matches data frame back to the (Meise) collectors and Wikidata items …

In [17]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,56,http://www.botanicalcollections.be/specimen/BR...,1898-01-28 14:12:18.462,1840-01-01 00:00:00.000,1995-10-26 00:00:00.000,49511,A,"Mas, A.",1.16
1,A Longhridge,G.,NaN,asper,NaN,NaN,NaN,NaN,"A Longhridge, G.",1,http://www.botanicalcollections.be/specimen/BR...,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000,1946-09-24 00:00:00.000,33902,"A Longhridge, G.","Loveridge, A.",1.04
2,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000,1954-08-01 00:00:00.000,46443,"A Robyns, Br","Robyns, F.H.E.A.W.",0.92
3,A',NaN,NaN,NaN,NaN,NaN,NaN,NaN,A',1,http://www.botanicalcollections.be/specimen/BR...,1896-08-06 00:00:00.000,1896-08-06 00:00:00.000,1896-08-06 00:00:00.000,30735,A',"Mas, A.",1.16
4,A'hlbon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'hlbon,1,http://www.botanicalcollections.be/specimen/BR...,1864-05-01 00:00:00.000,1864-05-01 00:00:00.000,1864-05-01 00:00:00.000,50118,A'hlbon,"Ahlburg, H.",1.01


In [18]:
# append full name matches
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A,Buto,NaN,NaN,NaN,NaN,NaN,NaN,"A, Buto",1,http://www.botanicalcollections.be/specimen/BR...,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000,1928-09-06 00:00:00.000,8211,"A, Buto","Budington, A B",1.16
1,A,Carl G.,NaN,NaN,NaN,NaN,NaN,NaN,"A, Carl G.",1,http://www.botanicalcollections.be/specimen/BR...,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,1956-09-13 00:00:00.000,11403,"A, Carl G.","Fenner, Carl G",0.98
2,A,Scha,NaN,NaN,NaN,NaN,NaN,NaN,"A, Scha",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,3713,"A, Scha","Schabel, A",1.10
3,A,Sti,NaN,NaN,NaN,NaN,NaN,NaN,"A, Sti",1,http://www.botanicalcollections.be/specimen/BR...,1852-01-01 00:00:00.000,1852-01-01 00:00:00.000,1852-01-01 00:00:00.000,991,"A, Sti","Thompson, A Stigston",1.08
4,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,"A. Brabant, Lochenies",1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000,1892-01-01 00:00:00.000,10138,"A. Brabant, Lochenies","Lochenies, Georges",0.98


In [19]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
121,Aalto,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Aalto, M.",16,http://www.botanicalcollections.be/specimen/BR...,1976-07-05 11:12:00.000,1975-07-07 00:00:00.000,1976-09-09 00:00:00.000,7776,"Aalto, M.","Aalto, M.",0.0
128,Aas,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Aas, O.",1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,49303,"Aas, O.","Aas, O.",0.0
148,Abbas,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbas, A.",44,http://www.botanicalcollections.be/specimen/BR...,1934-10-15 01:39:18.621,1869-05-04 00:00:00.000,2001-08-10 00:00:00.000,10180,"Abbas, A.","Abbas, A.",0.0
152,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",1,http://www.botanicalcollections.be/specimen/BR...,1932-09-01 00:00:00.000,1932-09-01 00:00:00.000,1932-09-01 00:00:00.000,6084,"Abbe, E.C.","Abbe, E.C.",0.0
160,Abbott,A.T.D.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, A.T.D.",1,http://www.botanicalcollections.be/specimen/BR...,2010-01-07 00:00:00.000,2010-01-07 00:00:00.000,2010-01-07 00:00:00.000,23346,"Abbott, A.T.D.","Abbott, A.T.D.",0.0


In [20]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

Save the plain name matching results only ...

In [21]:
if not os.path.exists('data'):
    os.makedirs('data')

this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
    # "20230913"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_all_matches.to_csv(this_output_file)

print("Wrote plain name matches of collector names into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote plain name matches of collector names into data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_plain-names_20230913.csv (12782 kB)


### Merge and Aggregate Matched Data and WikiData’s

Review (TODO)
- evaluate time references: `eventDate` ~ `yob`, `wyb`—perhaps define a score value that could integrate all scores from properties we need for decision of the name matching (name distance, eventDate ~ year of birth/work year begin aso.)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [22]:
# merge now the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [43]:
print("Show some name match examples (e.g. «Louis…» matching various names) …")
for testname in ['Louis', 'Abbot']:
    criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith(testname))    
    this_table=collectors_matches_g1_merged_wikidata[criterion].get([
        # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
        # 'canonical_string_fullname', 
        'itemLabel', 'wikidata_link',
        'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod', 'wyb', 'wye'
    ]).sort_values(by=['namematch_distance'])
    print("# ---------------------------------------------\n# «%s…» as test name, %d collector names begin with:" % (testname, criterion.sum()))    
    display(this_table)

Show some name match examples (e.g. «Louis…» matching various names) …
# ---------------------------------------------
# «Louis…» as test name, 120 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
39763,9,http://www.botanicalcollections.be/specimen/BR...,Louis-Marie,Louis-Marie,0.00,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1934-08-03 00:00:00.000,1952-07-01 00:00:00.000,1896.0,1978.0,NaN,NaN
39773,2348,http://www.botanicalcollections.be/specimen/BR...,"Louis, A.M.","Louis, A.M.",0.00,Adriaan M. Louis,http://www.wikidata.org/wiki/Q21338327,1902-11-28 00:00:00.000,1995-05-04 00:00:00.000,1944.0,NaN,NaN,NaN
18289,1,http://www.botanicalcollections.be/specimen/BR...,"Louis, A.","Louis, A.",0.00,A. Louis,http://www.wikidata.org/wiki/Q33682458,1944-03-10 00:00:00.000,1944-03-10 00:00:00.000,NaN,NaN,NaN,NaN
66685,53,http://www.botanicalcollections.be/specimen/BR...,"Louis, Piemeisel Robert","Piemeisel, Robert Louis",0.33,Robert Louis Piemeisel,http://www.wikidata.org/wiki/Q108905224,1927-06-28 00:00:00.000,1927-11-16 00:00:00.000,1889.0,1972.0,NaN,NaN
39765,22,http://www.botanicalcollections.be/specimen/BR...,"Louis-Marie, P.",Louis-Marie,0.36,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1921-08-19 00:00:00.000,1935-08-07 00:00:00.000,1896.0,1978.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66276,4,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Felix Boon",Louis-Marie,1.11,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1918-05-05 00:00:00.000,1920-08-29 00:00:00.000,1896.0,1978.0,NaN,NaN
66275,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Elv Bodart",Louis-Marie,1.12,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1893-05-17 00:00:00.000,1893-05-17 00:00:00.000,1896.0,1978.0,NaN,NaN
66672,2,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Elvire Bodart","Maire, Louis",1.12,Louis Maire,http://www.wikidata.org/wiki/Q19950303,1913-07-20 00:00:00.000,1913-07-20 00:00:00.000,NaN,1885.0,NaN,NaN
66675,1,http://www.botanicalcollections.be/specimen/BR...,"Louis Magnel, Htte Megnel","Magnel, Louis Robert Etienne Eugene Henri",1.14,Louis Robert Etienne Eugene Henri Magnel,http://www.wikidata.org/wiki/Q113552345,1893-08-01 00:00:00.000,1893-08-01 00:00:00.000,1863.0,1930.0,NaN,NaN


# ---------------------------------------------
# «Abbot…» as test name, 6 collector names begin with:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
914,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, A.T.D.","Abbott, A.T.D.",0.00,A. T. D. Abbott,http://www.wikidata.org/wiki/Q117328147,2010-01-07 00:00:00.000,2010-01-07 00:00:00.000,1936.0,2013.0,NaN,NaN
919,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, W.L.","Abbott, W.L.",0.00,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,1922-02-26 00:00:00.000,1922-02-26 00:00:00.000,1860.0,1936.0,NaN,NaN
911,2,http://www.botanicalcollections.be/specimen/BR...,Abbott,"Abbott, G.",0.43,George Abbott,http://www.wikidata.org/wiki/Q47112598,1982-11-16 00:00:00.000,1982-11-16 00:00:00.000,NaN,NaN,NaN,NaN
916,2,http://www.botanicalcollections.be/specimen/BR...,"Abbott, R.","Abbott, R.J.",0.53,R.J. Abbott,http://www.wikidata.org/wiki/Q33660683,1995-01-27 00:00:00.000,1995-06-20 00:00:00.000,NaN,NaN,NaN,NaN
918,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, W.C.","Abbott, W.L.",0.70,William Louis Abbott,http://www.wikidata.org/wiki/Q635604,NaT,NaT,1860.0,1936.0,NaN,NaN
915,1,http://www.botanicalcollections.be/specimen/BR...,"Abbott, J. Richard","Abbott, J.R.",0.87,J. Richard Abbott,http://www.wikidata.org/wiki/Q18982386,1995-03-22 00:00:00.000,1995-03-22 00:00:00.000,1968.0,NaN,NaN,NaN


Aggregate data to get atomized listings of multiple resource name matches joining by “|” aso.

In [44]:
print('Group data by canonical names (abbreviated and full name):'
      ' multiple related WD items (e.g. Q1232456), item labels, year of birth, year of death')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
    print('Run %s:   Group by wiki data’s %s, and aggregate/join item(s), labels, yob, yod '
          'by “…|…”, add new columns “…_joined” ...' % (i + 1, wd_matching_column))
    wdata_joined_items_and_others = wikidata.groupby([wd_matching_column]).agg(
        items_joined = ('item', lambda x: '|'.join(x)),
        item_labels_joined = ('itemLabel', lambda x: '|'.join(x)),
        yob_joined = ('yob', lambda x: '|'.join([str(s) for s in list(x)]) ),
        yod_joined = ('yod', lambda x: '|'.join([str(s) for s in list(x)]) )
    ).reset_index()

    # print("Done. Show examples of items having multiple matching data «|» … ")
    # criterion = wdata_joined_items['items'].map(lambda x: '|' in x)
    # wdata_joined_items[criterion].head()

    print('Run %s:   Merge all based on namematch_resource_data, add item(s) data ...' % (i + 1))
    collectors_matches_g2 = pd.merge(
        collectors_matches_g1_merged_wikidata, wdata_joined_items_and_others,
        left_on='namematch_resource_data', right_on=wd_matching_column
        , suffixes=('__wikidata_merge', '__grp_by_items')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_group” ...' % (i + 1))
    collectors_matches_group = collectors_matches_g2 \
        if i == 0 \
        else pd.concat([collectors_matches_group, collectors_matches_g2], ignore_index = True)
    
print('Done')

Group data by canonical names (abbreviated and full name): multiple related WD items (e.g. Q1232456), item labels, year of birth, year of death
Run 1:   Group by wiki data’s canonical_string, and aggregate/join item(s), labels, yob, yod by “…|…”, add new columns “…_joined” ...
Run 1:   Merge all based on namematch_resource_data, add item(s) data ...
Run 1:   Build data frame “collectors_matches_group” ...
Run 2:   Group by wiki data’s canonical_string_fullname, and aggregate/join item(s), labels, yob, yod by “…|…”, add new columns “…_joined” ...
Run 2:   Merge all based on namematch_resource_data, add item(s) data ...
Run 2:   Build data frame “collectors_matches_group” ...
Done


In [45]:
print("Show examples of item_labels_joined having multiple matching data «|» … ")
criterion = collectors_matches_group['item_labels_joined'].map(lambda x: '|' in x)

collectors_matches_group[criterion].get([ # empty 
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'item_labels_joined', 'items_joined', 'yob_joined', 'yod_joined'
], default="...get: Are data empty or it has probably a wrong named column?")


Show examples of item_labels_joined having multiple matching data «|» … 


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,item_labels_joined,items_joined,yob_joined,yod_joined
64,1,http://www.botanicalcollections.be/specimen/BR...,A-J,"He, J.",1.22,Ju He|Jie He|Jian He|Jin He,http://www.wikidata.org/entity/Q21515610|http:...,1976.0|nan|nan|nan,nan|nan|nan|nan
65,1,http://www.botanicalcollections.be/specimen/BR...,A-J,"He, J.",1.22,Ju He|Jie He|Jian He|Jin He,http://www.wikidata.org/entity/Q21515610|http:...,1976.0|nan|nan|nan,nan|nan|nan|nan
66,1,http://www.botanicalcollections.be/specimen/BR...,A-J,"He, J.",1.22,Ju He|Jie He|Jian He|Jin He,http://www.wikidata.org/entity/Q21515610|http:...,1976.0|nan|nan|nan,nan|nan|nan|nan
67,1,http://www.botanicalcollections.be/specimen/BR...,A-J,"He, J.",1.22,Ju He|Jie He|Jian He|Jin He,http://www.wikidata.org/entity/Q21515610|http:...,1976.0|nan|nan|nan,nan|nan|nan|nan
68,5,http://www.botanicalcollections.be/specimen/BR...,He,"He, J.",0.66,Ju He|Jie He|Jian He|Jin He,http://www.wikidata.org/entity/Q21515610|http:...,1976.0|nan|nan|nan,nan|nan|nan|nan
...,...,...,...,...,...,...,...,...,...
70985,4,http://www.botanicalcollections.be/specimen/BR...,"Walker, Jonathan S.","Walker, John",0.88,John Walker|John Walker,http://www.wikidata.org/entity/Q1588306|http:/...,1731.0|1930.0,1803.0|nan
70986,1,http://www.botanicalcollections.be/specimen/BR...,"Walker, MrSS B.","Walker, John",1.01,John Walker|John Walker,http://www.wikidata.org/entity/Q1588306|http:/...,1731.0|1930.0,1803.0|nan
70987,1,http://www.botanicalcollections.be/specimen/BR...,"Walker, MrSS B.","Walker, John",1.01,John Walker|John Walker,http://www.wikidata.org/entity/Q1588306|http:/...,1731.0|1930.0,1803.0|nan
71095,1,http://www.botanicalcollections.be/specimen/BR...,"White, David","White, David",0.00,David White|David White,http://www.wikidata.org/entity/Q5241020|http:/...,1862.0|nan,1935.0|nan


In [46]:
# check what columns we have and what we would keep for further analysis and what to drop
pprint.pprint(collectors_matches_group.columns)
# from merge: _x would means normally from left column, _y means from right column
# in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@#  @'

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'collectors_eventDate_mean', 'collectors_eventDate_min',
       'collectors_eventDate_max', 'old_index', 'namematch_source_data',
       'namematch_resource_data', 'namematch_distance', 'item', 'itemLabel',
       'surname', 'initials', 'canonical_string__wikidata_merge',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link',
       'canonical_string__grp_by_items', 'items_joined', 'item_labels_joined',
       'yob_joined', 'yod_joined', 'canonical_string',
       'canonical_string_fullname__wikidata_merge',
       'canonical_string_fullname__grp_by_items'],
      dtype='object')


Prepare data to save later on …

In [47]:
# Remove superfluous columns TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# TODO check duplicates
collectors_matches_group_simplified = collectors_matches_group.get(
    ['family', 'given', 'canonical_string_collector_parsed', 
      'namematch_source_data', # redundant: 'namematch_source_data' == 'canonical_string_collector_parsed'
      'namematch_resource_data', 'namematch_distance', 
      'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max', # collecors dates
      'yob_joined', 'yod_joined', # WikiData dates
      'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
      'items_joined', 'canonical_string', 'canonical_string_fullname', 'surname', 'initials', 'item_labels_joined'
    ], default="...get: Are data empty or it has probably a wrong named column?"
)
# collectors_matches_group = collectors_matches_g3
collectors_matches_group_simplified.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_group_simplified.drop_duplicates(inplace=True)
collectors_matches_group_simplified.head()

/tmp/ipykernel_4279/251392293.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.sort_values(
/tmp/ipykernel_4279/251392293.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.drop_duplicates(inplace=True)


,family,given,canonical_string_collector_parsed,namematch_source_data,namematch_resource_data,namematch_distance,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,yob_joined,...,harv,ipni,abbr,bionomia_id,items_joined,canonical_string,canonical_string_fullname,surname,initials,item_labels_joined
810,Aalto,M.,"Aalto, M.","Aalto, M.","Aalto, M.",0.0,1976-07-05 11:12:00.000,1975-07-07 00:00:00.000,1976-09-09 00:00:00.000,1939.0,...,6655,30583-1,Aalto,NaN,http://www.wikidata.org/entity/Q964994,NaN,"Aalto, Marjatta",Aalto,M.,Marjatta Aalto
825,Aas,O.,"Aas, O.","Aas, O.","Aas, O.",0.0,NaT,NaT,NaT,nan,...,74987,30584-1,Aas,NaN,http://www.wikidata.org/entity/Q36646127,NaN,"Aas, Olav",Aas,O.,Olav Aas
898,Abbas,A.,"Abbas, A.","Abbas, A.","Abbas, A.",0.0,1934-10-15 01:39:18.621,1869-05-04 00:00:00.000,2001-08-10 00:00:00.000,nan|nan,...,NaN,20034668-1,Al.Abbas,NaN,http://www.wikidata.org/entity/Q60141229|http:...,NaN,"Abbas, Alia",Abbas,A.,Alia Abbas|Abdulla Abbas
899,Abbas,A.,"Abbas, A.","Abbas, A.","Abbas, A.",0.0,1934-10-15 01:39:18.621,1869-05-04 00:00:00.000,2001-08-10 00:00:00.000,nan|nan,...,NaN,20034420-1,A.Abbas,NaN,http://www.wikidata.org/entity/Q60141229|http:...,NaN,"Abbas, Abdulla",Abbas,A.,Alia Abbas|Abdulla Abbas
903,Abbe,E.C.,"Abbe, E.C.","Abbe, E.C.","Abbe, E.C.",0.0,1932-09-01 00:00:00.000,1932-09-01 00:00:00.000,1932-09-01 00:00:00.000,1905.0,...,30066,26-1,Abbe,Q10274118,http://www.wikidata.org/entity/Q10274118,NaN,"Abbe, Ernst Cleveland",Abbe,E.C.,Ernst Cleveland Abbe


In [48]:
# old file meise_collectors_matches_wikidata_items_group_concat_%s.csv
this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_%s.csv' % (
    # "20230913"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_matches_group.to_csv(this_output_file)

print("Wrote groups of collectors matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote groups of collectors matches into data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_20230913.csv (38829 kB)


### Merge Data to Individual WikiData Items

For this, merge by namematch_resource_data and focus to get individual WikiData items.

In [49]:
print('Merge simply namematch_resource_data to Wiki data for abbreviated and full names... ')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):

    # join wikidata items to avh collectors matches
    #   avh_matches = pd.merge(avh, matches, left_on='label', right_on='name')
    #   avh_matches_t1 = pd.merge(avh_matches, wikidata, left_on='matched_name', right_on='canonical_string')
    # link counts of wikidata items with same canonical name string
    #   avh_matches_t2 = pd.merge(avh_matches_t1, wd_test, left_on="matched_name", right_on="canonical_string")
    #   avh_matches_t2.rename(columns = {list(avh_matches_t2.columns)[-1]: 'dup_count'}, inplace=True)
    
    print('Run %s:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...' % (i + 1))
    collectors_matches_wd1 = pd.merge(
        collectors_all_matches, wikidata,
        left_on='namematch_resource_data', right_on=wd_matching_column,
        suffixes=('__coll_all_matches', '__wd')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_with_wdata” ...' % (i + 1))
    collectors_matches_with_wdata = collectors_matches_wd1 \
        if i == 0 \
        else pd.concat([collectors_matches_with_wdata, collectors_matches_wd1], ignore_index=True)

print('Done')


Merge simply namematch_resource_data to Wiki data for abbreviated and full names... 
Run 1:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...
Run 1:   Build data frame “collectors_matches_with_wdata” ...
Run 2:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...
Run 2:   Build data frame “collectors_matches_with_wdata” ...
Done


In [50]:
pprint.pprint(collectors_matches_with_wdata.columns)
# echo "${text}" | fold --spaces | sed 's@^@#  @'

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'collectors_eventDate_mean', 'collectors_eventDate_min',
       'collectors_eventDate_max', 'old_index', 'namematch_source_data',
       'namematch_resource_data', 'namematch_distance', 'item', 'itemLabel',
       'surname', 'initials', 'canonical_string', 'canonical_string_fullname',
       'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob',
       'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 'harv_link',
       'ipni_link', 'bionomia_link'],
      dtype='object')


In [51]:
collectors_matches_with_wdata.drop_duplicates(inplace=True)
collectors_matches_with_wdata

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,Aalto,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Aalto, M.",16,...,NaN,1939.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q964994,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/30583-1,NaN
1,Nualto,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Nualto, J.",1,...,NaN,1939.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q964994,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/30583-1,NaN
2,Aas,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Aas, O.",1,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q36646127,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/30584-1,NaN
3,Aasmoerfh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aasmoerfh,1,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q36646127,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/30584-1,NaN
4,Abbas,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbas, A.",44,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q60141229,NaN,NaN,https://www.ipni.org/a/20034668-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71349,Milieuinventarisatie,Medewerkers Prov,NaN,NaN,NaN,NaN,NaN,NaN,"Milieuinventarisatie, Medewerkers Prov",7,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q36558917,https://orcid.org/0000-0001-8109-4544,NaN,https://www.ipni.org/a/20029314-1,NaN
71350,Pavia,Lab Crypt,NaN,NaN,NaN,NaN,NaN,NaN,"Pavia, Lab Crypt",1,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q88815659,NaN,NaN,https://www.ipni.org/a/20040640-1,NaN
71351,Plantenwerkgroep Wellemeersen,Jan Foselle,NaN,in Ruysseveldt Hugo in,NaN,NaN,NaN,NaN,"Plantenwerkgroep Wellemeersen, Jan Foselle",1,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q36558361,NaN,NaN,https://www.ipni.org/a/20021279-1,NaN
71352,Tanganika,Exporation Hydrologique Lac,NaN,NaN,NaN,NaN,NaN,NaN,"Tanganika, Exporation Hydrologique Lac",3,...,Q587674,1869.0,1953.0,NaN,NaN,http://www.wikidata.org/wiki/Q587674,NaN,https://kiki.huh.harvard.edu/databases/botanis...,NaN,https://bionomia.net/Q587674


Save all columns for further analysis

In [52]:
# old meise_collectors_matches_wikidata-botanists_all-columns_%s.csv

this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv' % (
    # "20230913"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_matches_with_wdata.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_with_wdata.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items of collector matches into data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_20230913.csv (31426 kB)


In [53]:
# TODO meaningful?
# remove redundant (duplicate (?or empty?)) columns that in any kind are duplicate data (i.e. that we usually do not need)
# do it by transposing it (https://www.statology.org/pandas-drop-duplicate-columns/)
compact_df_tmp=collectors_matches_with_wdata.transpose().drop_duplicates().transpose()
compact_df_tmp.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)

# old meise_collectors_matches_wikidata-botanists_all-columns-made-unique_%s.csv
# results_meise_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv
this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_%s.csv' % (
    # "20230705"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

compact_df_tmp.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items (unique columns) of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items (unique columns) of collector matches into data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_20230913.csv (30373 kB)


In [36]:
# TODO further evaluation or filtering, counting, clean up aso.

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
eventDate | date of the sampling event (required by GBIF, see https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
namematch_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest